In [ ]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import AutoTokenizer, AutoModel, get_scheduler
from torch.optim import AdamW
from sklearn.metrics import classification_report, f1_score,confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import json
import numpy as np
from tqdm import tqdm
from typing import List
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
# --- Constants ---
MODEL_NAME = 'roberta-base'
NUM_LABELS = 3
LABELS = ["No", "To some extent", "Yes"]
LABEL2ID = {label: i for i, label in enumerate(LABELS)}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}
TASKS = ['Mistake_Identification', 'Mistake_Location', 'Providing_Guidance', 'Actionability']

# --- Dataset Loader ---
class TutorDataset(Dataset):
    def __init__(self, data: List[dict], tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['conversation_history'] + "\nTutor Response: " + item['response']
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        labels = {task: LABEL2ID[item['annotation'][task]] for task in TASKS}

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor([labels[task] for task in TASKS], dtype=torch.long)
        }

# --- Model Definition ---
class PedagogicalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(0.2)
        self.heads = nn.ModuleDict({task: nn.Linear(self.encoder.config.hidden_size, NUM_LABELS) for task in TASKS})

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.dropout(outputs.last_hidden_state[:, 0])
        return {task: head(pooled) for task, head in self.heads.items()}

# --- Focal Loss ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

# --- Evaluation ---
def evaluate_model(model, dataloader):
    model.eval()
    device = next(model.parameters()).device
    y_true = {task: [] for task in TASKS}
    y_pred = {task: [] for task in TASKS}

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model(input_ids, attention_mask)
            for i, task in enumerate(TASKS):
                logits = outputs[task].cpu()
                preds = torch.argmax(logits, dim=-1)
                y_pred[task].extend(preds.tolist())
                y_true[task].extend(labels[:, i].tolist())

    return y_true, y_pred


In [3]:
# --- MODIFIED: Load JSON and Separate Tutor Responses ---
def load_json_dataset_by_tutor(path):
    with open(path, 'r') as f:
        raw = json.load(f)

    tutor_data = {}
    for conv in raw:
        for tutor_name, info in conv['tutor_responses'].items():
            sample = {
                'conversation_id': conv['conversation_id'],
                'conversation_history': conv['conversation_history'],
                'response': info['response'],
                'annotation': info['annotation']
            }
            tutor_data.setdefault(tutor_name, []).append(sample)
    return tutor_data

# --- Evaluate Predictions: Exact & Lenient ---
def evaluate_predictions(y_pred, y_true):
    for task in TASKS:
        print(f"\n=== {task} ===")
        print("-- Exact (3-Class) --")
        print(classification_report(
            y_true[task],
            y_pred[task],
            labels=[0, 1, 2],
            target_names=LABELS,
            digits=3
        ))
        print("Confusion Matrix:")
        print(confusion_matrix(y_true[task], y_pred[task], labels=[0, 1, 2]))
        print("Accuracy:", accuracy_score(y_true[task], y_pred[task]))
        print("Macro F1:", f1_score(y_true[task], y_pred[task], average='macro'))
        print("Micro F1:", f1_score(y_true[task], y_pred[task], average='micro'))
        # Lenient Evaluation (2-Class)
        def binarize(labels):
            return [0 if l == 0 else 1 for l in labels]  # No = 0, TSE/Yes = 1

        y_true_bin = binarize(y_true[task])
        y_pred_bin = binarize(y_pred[task])

        print("-- Lenient (2-Class) --")
        print(classification_report(
            y_true_bin,
            y_pred_bin,
            labels=[0, 1],
            target_names=["No", "Yes/TSE"],
            digits=3
        ))
        print("Confusion Matrix:")
        print(confusion_matrix(y_true_bin, y_pred_bin, labels=[0, 1]))
        print("Accuracy:", accuracy_score(y_true_bin, y_pred_bin))
        print("Macro F1:", f1_score(y_true_bin, y_pred_bin, average='macro'))
        print("Micro F1:", f1_score(y_true_bin, y_pred_bin, average='micro'))

# --- Training Function ---
def train_model(model, train_loader, val_loader, epochs=25, lr=2e-5, use_focal=True, save_dir="models"):
    import os
    os.makedirs(save_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)

    loss_fn = FocalLoss() if use_focal else nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = sum([loss_fn(outputs[task], labels[:, i]) for i, task in enumerate(TASKS)])
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

        # 🔒 Save model after each epoch
        torch.save(model.state_dict(), os.path.join(save_dir, f"model.pt"))

    # Evaluate after all epochs (once per tutor)
    y_true, y_pred = evaluate_model(model, val_loader)
    print("\nFINAL EVALUATION ON VALIDATION SET")
    evaluate_predictions(y_pred, y_true)
    return y_true, y_pred

In [ ]:
# --- Tutor-wise Training & Evaluation ---
if __name__ == '__main__':
    from sklearn.model_selection import train_test_split

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tutor_data = load_json_dataset_by_tutor('ai_tutors_dataset.json')
    tutor_scores = {}

    for tutor_name, data in tutor_data.items():
        print(f"\n\n================= TUTOR: {tutor_name} =================")

        train_data, val_data = train_test_split(data, test_size=0.3, random_state=42)
        train_ds = TutorDataset(train_data, tokenizer)
        val_ds = TutorDataset(val_data, tokenizer)

        train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=8)

        model = PedagogicalModel()
        train_model(model, train_loader, val_loader, epochs=50, lr=2e-5, use_focal=True)

        y_true, y_pred = evaluate_model(model, val_loader)
        tutor_scores[tutor_name] = {
            task: f1_score(y_true[task], y_pred[task], average='macro')
            for task in TASKS
        }

    # --- Display Summary ---
    print("\n\n========= SUMMARY: MACRO F1 SCORES BY TUTOR =========")
    for tutor, scores in tutor_scores.items():
        print(f"{tutor}:")
        for task in TASKS:
            print(f"  {task}: {scores[task]:.3f}")



================= TUTOR: Sonnet =================


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|██████████| 30/30 [00:13<00:00,  2.20it/s]


Epoch 1 Loss: 1.6083


Training Epoch 2: 100%|██████████| 30/30 [00:10<00:00,  2.86it/s]


Epoch 2 Loss: 1.4304


...

Epoch 48 Loss: 0.0082


Training Epoch 49: 100%|██████████| 8/8 [00:02<00:00,  3.24it/s]


Epoch 49 Loss: 0.0066


Training Epoch 50: 100%|██████████| 8/8 [00:02<00:00,  2.95it/s]


Epoch 50 Loss: 0.0063

FINAL EVALUATION ON VALIDATION SET

=== Mistake_Identification ===
-- Exact (3-Class) --
                precision    recall  f1-score   support

            No      0.000     0.000     0.000         2
To some extent      0.400     0.667     0.500         3
           Yes      0.889     0.727     0.800        11

      accuracy                          0.625        16
     macro avg      0.430     0.465     0.433        16
  weighted avg      0.686     0.625     0.644        16

Confusion Matrix:
[[0 2 0]
 [0 2 1]
 [2 1 8]]
Accuracy: 0.625
Macro F1: 0.4333333333333333
Micro F1: 0.625
-- Lenient (2-Class) --
              precision    recall  f1-score   support

          No      0.000     0.000     0.000         2
     Yes/TSE      0.857     0.857     0.857        14

    accuracy                          0.750        16
   macro avg      0.429     0.429     0.429        16
weighted avg      0.750     0.750     0.750        16

Confusion Matrix:
[[ 0  2]
 [ 2 12]

In [ ]:
# --- Tutor-wise Training & Evaluation ---
if __name__ == '__main__':
    from sklearn.model_selection import train_test_split

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tutor_data = load_json_dataset_by_tutor('ai_tutors_dataset.json')
    tutor_scores = {}

    for tutor_name, data in tutor_data.items():
        print(f"\n\n================= TUTOR: {tutor_name} =================")

        train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
        train_ds = TutorDataset(train_data, tokenizer)
        val_ds = TutorDataset(val_data, tokenizer)

        train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=8)

        model = PedagogicalModel()
        train_model(model, train_loader, val_loader, epochs=25, lr=2e-5, use_focal=True)

        y_true, y_pred = evaluate_model(model, val_loader)
        tutor_scores[tutor_name] = {
            task: f1_score(y_true[task], y_pred[task], average='macro')
            for task in TASKS
        }

    # --- Display Summary ---
    print("\n\n========= SUMMARY: MACRO F1 SCORES BY TUTOR =========")
    for tutor, scores in tutor_scores.items():
        print(f"{tutor}:")
        for task in TASKS:
            print(f"  {task}: {scores[task]:.3f}")




================= TUTOR: Sonnet =================


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Epoch 1: 100%|██████████| 30/30 [00:12<00:00,  2.46it/s]


Epoch 1 Loss: 1.5855


Training Epoch 2: 100%|██████████| 30/30 [00:12<00:00,  2.40it/s]


Epoch 2 Loss: 1.5351


...

Epoch 23 Loss: 0.1153


Training Epoch 24: 100%|██████████| 8/8 [00:01<00:00,  4.08it/s]


Epoch 24 Loss: 0.1143


Training Epoch 25: 100%|██████████| 8/8 [00:01<00:00,  4.07it/s]


Epoch 25 Loss: 0.1244

FINAL EVALUATION ON VALIDATION SET

=== Mistake_Identification ===
-- Exact (3-Class) --
                precision    recall  f1-score   support

            No      0.000     0.000     0.000         2
To some extent      0.286     0.667     0.400         3
           Yes      1.000     0.636     0.778        11

      accuracy                          0.562        16
     macro avg      0.429     0.434     0.393        16
  weighted avg      0.741     0.562     0.610        16

Confusion Matrix:
[[0 2 0]
 [1 2 0]
 [1 3 7]]
Accuracy: 0.5625
Macro F1: 0.3925925925925926
Micro F1: 0.5625
-- Lenient (2-Class) --
              precision    recall  f1-score   support

          No      0.000     0.000     0.000         2
     Yes/TSE      0.857     0.857     0.857        14

    accuracy                          0.750        16
   macro avg      0.429     0.429     0.429        16
weighted avg      0.750     0.750     0.750        16

Confusion Matrix:
[[ 0  2]
 [ 2 1